Importing the reqiured libraries

In [1]:
import librosa
import soundfile
import os,glob,pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score 

Feature extraction

In [2]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X= sound_file.read(dtype= "float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result= np.array([])
        if mfcc:
            mfccs= np.mean(librosa.feature.mfcc(y=X, sr=sample_rate,n_mfcc=40).T, axis=0)
            result= np.hstack((result,mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate,n_mfcc=40).T, axis=0)
            result= np.hstack((result,chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
            result= np.hstack((result,mel))
    return result

Defining a dictionary to hold numbers and the emotions 

In [66]:
emotions ={
    '01': 'neutral',
    '02' : 'calm',
    '03' : 'happy',
    '04' : 'sad',
    '05' : 'angry',
    '06' : 'fearful',
    '07' : 'disgust',
    '08' : 'surprised'
}
observed_emotions=['calm','happy','fearful','disgust']


Loading data using load_data() and using glob() function to get pathnames for the sound files

In [67]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("G:\\Project\\speech-emotion-recognition-ravdess-data\\Actor_*\\*.wav"):
        file_name= os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature= extract_feature(file,mfcc=True, chroma= True, mel= True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x),y, test_size=test_size, random_state= 9)

Spliting of dataset into training and test set (Test set is 25% of the whole dataset) using load_data function

In [81]:
x_train, x_test, y_train, y_test = load_data(test_size= 0.25)

In [82]:
print((x_train.shape[0], x_test.shape[0]))

(384, 384)


In [83]:
print(f'feature extracted:{x_train.shape[1]}')

feature extracted:208


Initializing a Multi-layer Perceptron Classifer

In [84]:
model = MLPClassifier(alpha=0.01,
                      batch_size=256, 
                      epsilon= 1e-08,
                      hidden_layer_sizes=(300,), 
                      learning_rate='adaptive',
                      max_iter=500)

Fit/train the model

In [85]:
model.fit(x_train, y_train)

C:\Users\chett\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_iter=500, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

Predicting values of the tets set

In [86]:
y_pred= model.predict(x_test)

In [87]:
y_pred

array(['happy', 'fearful', 'happy', 'happy', 'fearful', 'calm', 'disgust',
       'disgust', 'disgust', 'happy', 'fearful', 'fearful', 'fearful',
       'happy', 'disgust', 'happy', 'calm', 'happy', 'happy', 'fearful',
       'calm', 'disgust', 'disgust', 'calm', 'fearful', 'happy', 'happy',
       'happy', 'fearful', 'happy', 'happy', 'disgust', 'happy',
       'fearful', 'disgust', 'calm', 'calm', 'fearful', 'calm', 'calm',
       'happy', 'calm', 'calm', 'calm', 'fearful', 'calm', 'disgust',
       'disgust', 'calm', 'happy', 'fearful', 'fearful', 'fearful',
       'fearful', 'happy', 'fearful', 'disgust', 'happy', 'calm', 'calm',
       'disgust', 'calm', 'fearful', 'calm', 'happy', 'calm', 'calm',
       'disgust', 'fearful', 'happy', 'fearful', 'fearful', 'fearful',
       'fearful', 'fearful', 'disgust', 'fearful', 'happy', 'calm',
       'fearful', 'disgust', 'calm', 'fearful', 'calm', 'disgust', 'calm',
       'disgust', 'fearful', 'disgust', 'fearful', 'disgust', 'disgust',
 

Calculating the accuracy of the model

In [88]:
accuracy= accuracy_score(y_true=y_test, y_pred= y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 71.88%
